In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
_ = """
Although design patterns are language-independent, that does not mean every pattern
applies to every language. In his 1996 presentation, “Design Patterns in Dynamic Languages”,
Peter Norvig states that 16 out of the 23 patterns in the original Design Patterns
book by Gamma et al. become either “invisible or simpler” in a dynamic language (slide
9). He was talking about Lisp and Dylan, but many of the relevant dynamic features are
also present in Python
"""

strategy_pattern = """
Define a family of algorithms, encapsulate each one, and make them interchangeable.
Strategy lets the algorithm vary independently from clients that use it.

A clear example of Strategy applied in the ecommerce domain is computing discounts
to orders according to the attributes of the customer or inspection of the ordered items.
"""

In [5]:
# implementation of strategy pattern(Order)
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0


joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo())
Order(ann, cart, FidelityPromo())

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

<Order total: 42.00 due: 39.90>

<Order total: 30.00 due: 28.50>

<Order total: 10.00 due: 9.30>

<Order total: 42.00 due: 42.00>

In [6]:
import inspect

_ = """
but the same functionality can be implemented with less code in Python by using functions as objects.
replacing the concrete strategies with simple functions and removing the Promo abstract class.
no instance attributes
"""

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0


joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, fidelity_promo)
Order(ann, cart, fidelity_promo)

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, large_order_promo)
Order(joe, cart, large_order_promo)

# choose the best strategy
promos = [fidelity_promo, large_order_promo, bulk_item_promo]  # not ideal when adding new promotion


def best_promo(order):
    return max(promo(order) for promo in promos)


Order(joe, long_order, best_promo)
Order(joe, banana_cart, best_promo)
Order(ann, cart, best_promo)

# find strategies in a Module
_ = """
modules in Python are also first-class objects
globals()
Return a dictionary representing the current global symbol table. This is always the
dictionary of the current module (inside a function or method, this is the module
where it is defined, not the module from which it is called).
"""
promos = [globals()[name] for name in globals()
          if name.endswith('_promo') and name != 'best_promo']
promos


def best_promo(order):
    return max(promo(order) for promo in promos)


Order(joe, long_order, best_promo)
Order(joe, banana_cart, best_promo)
Order(ann, cart, best_promo)

# another way
promos = [func for name, func in
          inspect.getmembers(promotions, inspect.isfunction)]   # promotion is the new module
promos


<Order total: 42.00 due: 42.00>

<Order total: 42.00 due: 39.90>

<Order total: 30.00 due: 28.50>

<Order total: 10.00 due: 9.30>

<Order total: 42.00 due: 42.00>

...

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

<Order total: 10.00 due: 9.30>

<Order total: 30.00 due: 28.50>

<Order total: 42.00 due: 39.90>

NameError: name 'promotions' is not defined

In [7]:
# command pattern
_ = """
The goal of Command is to decouple an object that invokes an operation (the Invoker)
from the provider object that implements it (the Receiver). In the example from Design
Patterns, each invoker is a menu item in a graphical application, and the receivers are
the document being edited or the application itself.
The idea is to put a Command object between the two, implementing an interface with a
single method, execute, which calls some method in the Receiver to perform the desired
operation. That way the Invoker does not need to know the interface of the Receiver,
and different receivers can be adapted through different Command subclasses. The Invoker
is configured with a concrete command and calls its execute method to operate
it. Note in Figure 6-2 that MacroCommand may store a sequence of commands; its
execute() method calls the same method in each command stored.
"""
class MacroCommand:
    """A command that executes a list of commands"""
    def __init__(self, commands):
        self.commands = list(commands)
    
    def __call__(self):
        for command in self.commands:
            command()